In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras import backend as K

from CRNN import KerasCrnn

tf.keras.backend.clear_session() 

In [2]:
# IMG_HEIGHT = 1026    # number of freq bins (rows / pixels high) in each spectrogram
IMG_HEIGHT = 24
IMG_WIDTH = int(10e3)
N_LABELS = 28
BATCH_SIZE = 64
LSTM_SIZE = 64

In [7]:
def ctc_custom_len(args):
    _true_labels, _pred_labels, _label_lens = args
    pred_len = [[_pred_labels.shape[1].value]] * BATCH_SIZE
    pred_len = K.variable(pred_len)
    
    return K.ctc_batch_cost(_true_labels, K.softmax(_pred_labels), K.variable(pred_len, trainable=False), _label_lens)

In [8]:
# Define tensors for the waterfall inputs, label outputs, and predicted label outputs
true_labels = Input(name='feature_labels', shape=[N_LABELS], dtype='int32')
label_lens = Input(name='label_lens', shape=[1], dtype='int32')
imgs = Input(name='waterfall_input', shape=[IMG_HEIGHT, IMG_WIDTH, 1], dtype='float32')

In [9]:
# Construct CRNN architecture
crnn = KerasCrnn(IMG_HEIGHT, N_LABELS, nh=LSTM_SIZE)
pred_labels = crnn(imgs)

In [10]:
# Define loss
loss = Lambda(ctc_custom_len, output_shape=(0,), name='ctc')([true_labels, pred_labels, label_lens])

TypeError: Expected float32, got None of type '_Message' instead.